### Naive-Bayes Model

In [1]:
import numpy as np
import pandas as pd

np.set_printoptions(precision=4, suppress=True)

In [32]:
np.random.seed(4)

data = np.random.rand(10,11)
data = np.round(data, decimals=0)
X = data[:, :-1]
y = data[:, -1]

In [33]:
X.shape, y.shape

((10, 10), (10,))

class Naive_Bayes:
    def compute_Px1y1(self, X, y):
        d = X.shape[1]
        return (1/(np.sum(y==1))) * np.sum(np.sum(X[y==1], axis=1) == d)
    
    def compute_Px1y0(self, X, y):
        d = X.shape[1]
        return (1/(np.sum(y==0))) * np.sum(np.sum(X[y==0], axis=1) == d)
    
    def compute_Px0y0(self, X, y):
        return (1/(np.sum(y==0))) * np.sum(np.sum(X[y==0], axis=1) == 0)
    
    def compute_Px0y1(self, X, y):
        return (1/(np.sum(y==1))) * np.sum(np.sum(X[y==1], axis=1) == 0)
    
    def compute_Py1(self, y):
        n = len(y)
        return (1/n) * np.sum(y==1)
    
    #def compute_Pxy(self, X, idx):
        #return np.power(self.phi, 
    
    def fit(self, X, y):
        d = X.shape[1]
        self.Phix1y1 = self.compute_Px1y1(X, y)
        self.Phix1y0 = self.compute_Px1y0(X, y)
        self.Phix0y0 =  self.compute_Px0y0(X, y)
        self.Phix0y1 =  self.compute_Px0y0(X, y)
        self.Phiy1 = self.compute_Py1(y)
        self.Phiy0 = 1 - self.Phiy1

    def predict_proba(self, X):
        Pxy1 = (self.Phix1y1 + self.Phix0y1)
        Pxy0 = (self.Phix1y0 + self.Phix0y0)
        
        Pxy_0 = np.power(Pxy0, X)*np.power((1-Pxy0), (1-X))
        Pxy_0 = np.sum(Pxy_0, axis = 1)
        Py0 = Pxy_0 * self.Phiy0
        Py0 = Py0.reshape(-1, 1)
        
        Pxy_1 = np.power(Pxy1, X)*np.power((1-Pxy1), (1-X))
        Pxy_1 = np.sum(Pxy_1, axis = 1)
        Py1 = Pxy_1 * self.Phiy1
        Py1 = Py1.reshape(-1, 1)
                        
        return np.concatenate((Py0, Py1), axis=1)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)

In [48]:
class Naive_Bayes:
    def compute_Px1y1(self, X, y):
        return np.sum(X[y==1] == 1) / np.sum(y==1)
    
    def compute_Px1y0(self, X, y):
        return np.sum(X[y==0] == 1) / np.sum(y==0)
    
    def compute_Px0y0(self, X, y):
        return np.sum(X[y==0] == 0) / np.sum(y==0)
    
    def compute_Px0y1(self, X, y):
        return np.sum(X[y==1] == 0) / np.sum(y==1)
    
    def compute_Py1(self, y):
        n = len(y)
        return (1/n) * np.sum(y==1)
    
    def compute_Py0(self, y):
        n = len(y)
        return (1/n) * np.sum(y==0)
    
    #def compute_Pxy(self, X, idx):
        #return np.power(self.phi, 
    
    def fit(self, X, y):
        d = X.shape[1]
        PhiXy_dict = dict()
        for idx in range(d):
            PhiXy_dict[idx] = dict()
            Phix1y1 = self.compute_Px1y1(X[:, idx], y)
            Phix1y0 = self.compute_Px1y0(X[:, idx], y)
            Phix0y0 =  self.compute_Px0y0(X[:, idx], y)
            Phix0y1 =  self.compute_Px0y1(X[:, idx], y)
            PhiXy_dict[idx].update({
                            'Phix0y0':Phix0y0, 
                            'Phix0y1':Phix0y1,
                            'Phix1y0':Phix1y0,
                            'Phix1y1':Phix1y1
                            })
        print(PhiXy_dict)
        self.phiXy = PhiXy_dict                  
        self.Phiy1 = self.compute_Py1(y)
        self.Phiy0 = self.compute_Py0(y)

    def predict_proba(self, X):
        Pxy0 = np.zeros(X.shape)
        Pxy1 = np.zeros(X.shape)
        n, d = X.shape
        
        for row in range(n):
            for col in range(d):
                if X[row, col] == 1:
                    Pxy0[row, col] = self.phiXy[col]['Phix1y0']
                    Pxy1[row, col] = self.phiXy[col]['Phix1y1']
                else:
                    Pxy0[row, col] = self.phiXy[col]['Phix0y0']
                    Pxy1[row, col] = self.phiXy[col]['Phix0y1']
        
        Pxy1 = np.multiply.reduce(Pxy1, axis=1)
        Pxy0 = np.multiply.reduce(Pxy0, axis=1)
        
        Py0 = Pxy0 * self.Phiy0
        Py0 = Py0.reshape(-1, 1)
        
        Py1 = Pxy1 * self.Phiy1
        Py1 = Py1.reshape(-1, 1)
                        
        return np.concatenate((Py0, Py1), axis=1)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)

In [51]:
class Naive_Bayes:
    def compute_Px1y1(self, X, y):
        return np.sum(X[y==1] == 1) / np.sum(y==1)
    
    def compute_Px1y0(self, X, y):
        return np.sum(X[y==0] == 1) / np.sum(y==0)
    
    def compute_Px0y0(self, X, y):
        return np.sum(X[y==0] == 0) / np.sum(y==0)
    
    def compute_Px0y1(self, X, y):
        return np.sum(X[y==1] == 0) / np.sum(y==1)
    
    def compute_Py1(self, y):
        n = len(y)
        return (1/n) * np.sum(y==1)
    
    def compute_Py0(self, y):
        n = len(y)
        return (1/n) * np.sum(y==0)
    
    #def compute_Pxy(self, X, idx):
        #return np.power(self.phi, 
    
    def fit(self, X, y):
        d = X.shape[1]
        PhiXy_arr = []
        for idx in range(d):
            PhiXy = np.zeros((2,2))
            PhiXy[1,1] = self.compute_Px1y1(X[:, idx], y)
            PhiXy[1,0] = self.compute_Px1y0(X[:, idx], y)
            PhiXy[0,0] =  self.compute_Px0y0(X[:, idx], y)
            PhiXy[0,1] =  self.compute_Px0y1(X[:, idx], y)
            
            PhiXy_arr.append(PhiXy)
        PhiXy_arr = np.array(PhiXy_arr)
        #print(PhiXy_arr)
        self.phiXy = PhiXy_arr                  
        self.Phiy1 = self.compute_Py1(y)
        self.Phiy0 = self.compute_Py0(y)

    def predict_proba(self, X):
        Pxy0 = np.zeros(X.shape)
        Pxy1 = np.zeros(X.shape)
        n, d = X.shape
        
        for row in range(n):
            for col in range(d):
                if X[row, col] == 1:
                    Pxy0[row, col] = self.phiXy[col][1][0]
                    Pxy1[row, col] = self.phiXy[col][1][1]
                else:
                    Pxy0[row, col] = self.phiXy[col][0][0]
                    Pxy1[row, col] = self.phiXy[col][0][1]
        
        Pxy1_i = np.multiply.reduce(Pxy1, axis=1)
        Pxy0_i = np.multiply.reduce(Pxy0, axis=1)
        
        Py0 = Pxy0_i * self.Phiy0
        Py0 = Py0.reshape(-1, 1)
        
        Py1 = Pxy1_i * self.Phiy1
        Py1 = Py1.reshape(-1, 1)
                        
        return np.concatenate((Py0, Py1), axis=1)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        return np.argmax(proba, axis=1)

In [52]:
nb = Naive_Bayes()

In [53]:
nb.fit(X, y)

[[[0.6667 0.4286]
  [0.3333 0.5714]]

 [[0.6667 0.2857]
  [0.3333 0.7143]]

 [[0.     0.4286]
  [1.     0.5714]]

 [[0.6667 0.7143]
  [0.3333 0.2857]]

 [[0.     0.4286]
  [1.     0.5714]]

 [[0.3333 0.4286]
  [0.6667 0.5714]]

 [[0.     0.4286]
  [1.     0.5714]]

 [[0.6667 0.8571]
  [0.3333 0.1429]]

 [[0.3333 0.2857]
  [0.6667 0.7143]]

 [[0.3333 0.5714]
  [0.6667 0.4286]]]


In [54]:
predictions = nb.predict(X)

In [55]:
predictions

array([1, 1, 1, 0, 1, 1, 0, 1, 1, 1])

In [56]:
np.sum(predictions == y) / len(y)

0.9

In [57]:
nb.predict_proba(X)

array([[0.0003, 0.0009],
       [0.    , 0.0032],
       [0.    , 0.0032],
       [0.0044, 0.0005],
       [0.0044, 0.0057],
       [0.    , 0.0007],
       [0.0022, 0.0002],
       [0.    , 0.0004],
       [0.    , 0.0017],
       [0.    , 0.0013]])